# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

print(geoapify_key)

8aa2cf2663c24699bde19c89c7bcd6ba


In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/ryanpope/python-api-challenge/Starter_Code/output_data/newcities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,275.29,76,68,8.09,CL,1727232128
1,1,olonkinbyen,70.9221,-8.7187,274.13,78,36,7.65,SJ,1727232243
2,2,waitangi,-43.9535,-176.5597,282.49,77,28,6.26,NZ,1727232244
3,3,blackmans bay,-43.0167,147.3167,286.33,49,100,0.89,AU,1727232244
4,4,bilibino,68.0546,166.4372,272.86,66,100,0.61,RU,1727232244


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
# Configure the map plot using hvplot with humidity as the size of the points
city_data_df.hvplot.points(
    'Lng', 'Lat',  # Use 'Lng' for longitude and 'Lat' for latitude
    geo=True,      # Enable geographic plotting
    tiles='OSM',   # Use OpenStreetMap tiles as the background
    color='City',  # Color by the city (or any other relevant column)
    size='Humidity',  # Use 'Humidity' column to adjust the size of the points
    frame_width=800, frame_height=500,  # Set the plot size
    hover_cols=['City', 'Humidity']  # Add hover tool to display the city names and humidity
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [82]:
# Step 1: Convert Max Temp from Kelvin to Fahrenheit
city_data_df['Max Temp (F)'] = (city_data_df['Max Temp'] - 273.15) * 9/5 + 32  # Corrected to +32 for Fahrenheit conversion

# Step 2: Print summary statistics to check the current range of temperatures in Fahrenheit
print("Summary statistics of Max Temp (F) before scaling:")
print(city_data_df['Max Temp (F)'].describe())

# Step 3: Find min and max values of the Fahrenheit temperature
min_fahrenheit = city_data_df['Max Temp (F)'].min()
max_fahrenheit = city_data_df['Max Temp (F)'].max()



# Step 6: Display sample data to verify the transformation
city_data_df[['City', 'Max Temp (F)']].head()


Summary statistics of Max Temp (F) before scaling:
count    577.00000
mean      66.11992
std       15.31574
min       22.83800
25%       55.11200
50%       68.45000
75%       78.65600
max      100.85000
Name: Max Temp (F), dtype: float64


,City,Max Temp (F)
0,puerto natales,35.852
1,olonkinbyen,33.764
2,waitangi,48.812
3,blackmans bay,55.724
4,bilibino,31.478


In [108]:
    # Define less restrictive criteria based on Max Temp (F), Latitude, Longitude, and Humidity
    new_criteria = (
        (city_data_df['Max Temp (F)'] >= 50) & (city_data_df['Max Temp (F)'] <= 70) &
        (city_data_df['Lat'] >= -40) & (city_data_df['Lat'] <= 55) &
        ((city_data_df['Lat'] < 20) | (city_data_df['Lat'] > -20)) & 
        (city_data_df['Humidity'] > 70) & (city_data_df['Humidity'] < 95)
    )

# Apply the criteria to filter the DataFrame
filtered_city_data_df = city_data_df[new_criteria]

# Drop rows with missing values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display the filtered DataFrame sample
filtered_city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (F),Max Temp (F) (Scaled)
5,5,mar del plata,-38.0023,-57.5575,285.90,82,95,4.47,AR,1727231789,54.950,37.046608
11,11,gijon,43.5357,-5.6615,291.18,91,100,0.45,ES,1727232222,64.454,48.011075
12,12,bairnsdale,-37.8333,147.6167,285.96,89,100,4.02,AU,1727232244,55.058,37.171204
13,13,turkmenbasy,40.0222,52.9552,293.58,77,20,5.14,TM,1727232245,68.774,52.994924
17,17,bethel,41.3712,-73.4140,289.23,83,75,2.06,US,1727232121,60.944,43.961698


### Step 3: Create a new DataFrame called `hotel_df`.

In [109]:
# Step 1: Use the Pandas copy function to create a new DataFrame
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Step 2: Add an empty column for storing the hotel names
hotel_df['Hotel Name'] = ""

# Step 3: Display the sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
5,mar del plata,AR,-38.0023,-57.5575,82,
11,gijon,ES,43.5357,-5.6615,91,
12,bairnsdale,AU,-37.8333,147.6167,89,
13,turkmenbasy,TM,40.0222,52.9552,77,
17,bethel,US,41.3712,-73.4140,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [110]:
import requests

import time


# Step 1: Use the Pandas copy function to create a new DataFrame for storing hotel data
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Step 2: Add an empty column for storing the hotel names
hotel_df['Hotel Name'] = ""

# Step 3: Display the sample data
print("Initial data (first 5 rows):")
print(hotel_df.head())

# Step 4: Define the fixed search radius (10,000 meters) and API base URL
radius = 10000
base_url = "https://api.geoapify.com/v2/places"
print("Starting hotel search...")

# Step 5: Iterate through each city in the DataFrame and fetch the nearest hotel data
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Ensure valid latitude and longitude
    if not (-90 <= lat <= 90) or not (-180 <= lng <= 180):
        print(f"Invalid coordinates for {row['City']} (Lat: {lat}, Lng: {lng}). Skipping...")
        hotel_df.loc[index, "Hotel Name"] = "Invalid coordinates"
        continue
    
    # Set the parameters for the Geoapify API request
    params = {
        "categories": "accommodation.hotel",  # Looking for hotels
        "filter": f"circle:{lng},{lat},{radius}",  # Fixed radius of 10,000 meters
        "bias": f"proximity:{lng},{lat}",  # Bias results toward proximity of coordinates
        "limit": 1,  # Limit results to the first hotel
        "apiKey": geoapify_key  # Use the imported Geoapify API key
    }

    # Make the API request and handle potential request errors
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an error for bad responses
        name_address = response.json()

        # Debugging: Print the entire response if the expected data is missing
        if "features" not in name_address or len(name_address["features"]) == 0:
            print(f"No hotel found for {row['City']} within a 10,000-meter radius. Full response: {name_address}")
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            continue

        # Extract the hotel name from the response
        hotel_name = name_address["features"][0]["properties"].get("name", "Unnamed hotel")
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"Hotel found for {row['City']}: {hotel_name}")

    except requests.RequestException as e:
        print(f"Error fetching hotel data for {row['City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

# Display sample data
hotel_df


Initial data (first 5 rows):
             City Country      Lat       Lng  Humidity Hotel Name
5   mar del plata      AR -38.0023  -57.5575        82           
11          gijon      ES  43.5357   -5.6615        91           
12     bairnsdale      AU -37.8333  147.6167        89           
13    turkmenbasy      TM  40.0222   52.9552        77           
17         bethel      US  41.3712  -73.4140        83           
Starting hotel search...
Hotel found for mar del plata: Hotel 3 de Abril
Hotel found for gijon: Hotel Begoña Centro
Hotel found for bairnsdale: Main hotel
Hotel found for turkmenbasy: Hazar Myhmanhanasy
Hotel found for bethel: Hampton Inn Danbury
Hotel found for porto-vecchio: Unnamed hotel
No hotel found for adamstown within a 10,000-meter radius. Full response: {'type': 'FeatureCollection', 'features': []}
Hotel found for antigonish: Microtel Hotel
Hotel found for colonia: Wasserturm Hotel Cologne
Hotel found for hermanus: Aloe guest house
Hotel found for borkum: Fer

,City,Country,Lat,Lng,Humidity,Hotel Name
5,mar del plata,AR,-38.0023,-57.5575,82,Hotel 3 de Abril
11,gijon,ES,43.5357,-5.6615,91,Hotel Begoña Centro
12,bairnsdale,AU,-37.8333,147.6167,89,Main hotel
13,turkmenbasy,TM,40.0222,52.9552,77,Hazar Myhmanhanasy
17,bethel,US,41.3712,-73.4140,83,Hampton Inn Danbury
...,...,...,...,...,...,...
551,caleta de carquin,PE,-11.0925,-77.6267,90,Hotel Mi Choacan
552,bafra,TR,41.5678,35.9069,78,No hotel found
554,whittlesea,ZA,-32.1759,26.8244,89,No hotel found
570,taveta,KE,-3.3963,37.6736,81,Garden hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map

In [111]:
# Configure the map plot using hvplot with humidity as the size of the points
hotel_df.hvplot.points(
    'Lng', 'Lat',  # Use 'Lng' for longitude and 'Lat' for latitude
    geo=True,      # Enable geographic plotting
    tiles='OSM',   # Use OpenStreetMap tiles as the background
    color='City',  # Color by the city (or any other relevant column)
    size='Humidity',  # Use 'Humidity' column to adjust the size of the points
    frame_width=800, frame_height=500,  # Set the plot size
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']  # Add hover tool to display city, country, hotel name, and humidity
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)